# Apply document classifier

This script applies the trained classifier to scraped documents.

In [1]:
import pandas as pd
import os
import spacy
from spacy.util import minibatch, compounding

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


## Import texts

In [2]:
texts_scraped = pd.read_csv(os.path.join(os.getcwd(), '..', '..', 'data', 'text_scraped.csv'))
texts = texts_scraped['text']
texts.head()

0    Sands CISD District of Innovation Plan 2018-20...
1    Ricardo Independent School District Final Dist...
2                                              No text
3    Gold-Burg Independent School District District...
4    Joaquin Independent School District District o...
Name: text, dtype: object

## Import and apply model

In [3]:
# Load model
model_dir = os.path.join(os.getcwd(),  '..', '..', 'data', 'document_classifier')
# apply the saved model
print("Loading from", model_dir)
nlp = spacy.load(model_dir)
categories = []
for text in texts:
    doc = nlp(text)
    categories.append(doc.cats)

Loading from /Users/kylieleblancKylie/dofis/code/exemptions/../../data/document_classifier


/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [4]:
p_innovation = [x['INNOVATION'] for x in categories]

In [5]:
texts_scraped_p = texts_scraped
texts_scraped_p['p_innovation'] = p_innovation
texts_scraped_p.tail()

,Unnamed: 0,Unnamed: 0.1,title,level,type,link,text,p_innovation
3731,3731,390,Kerrville ISD,html,html,https://www.kerrvilleisd.net/domain/41,District of Innovation / District of Innovatio...,0.011909
3732,3732,391,Mineola ISD,html,html,http://www.mineolaisd.net/page/District-of-Inn...,Mineola Independent School District Skip Navig...,0.148307
3733,3733,392,Leary ISD,html,html,http://www.learyisd.net/6253_1,Leary ISD - Public Info Skip to main content W...,0.008536
3734,3734,393,Goliad ISD,html,html,http://www.goliadisd.org/apps/pages/index.jsp?...,District of Innovation – Plans – Goliad Indepe...,0.089283
3735,3735,394,Anahuac ISD,html,html,https://sites.google.com/aisdpanthers.com/anah...,Anahuac ISD - District of Innovation Search th...,0.002121


In [6]:
texts_scraped_p.head()

,Unnamed: 0,Unnamed: 0.1,title,level,type,link,text,p_innovation
0,0,0,Sands CISD,First,pdf,http://sands.esc17.net/upload/page/0019/docs/S...,Sands CISD District of Innovation Plan 2018-20...,0.999955
1,1,1,Ricardo ISD,First,pdf,http://www.ricardoisd.us/UserFiles/Servers/Ser...,Ricardo Independent School District Final Dist...,0.998646
2,2,2,Stanton ISD,First,pdf,http://www.stanton.esc18.net/site/handlers/fil...,No text,0.001259
3,3,3,Gold-Burg ISD,First,pdf,http://images.pcmac.org/SiSFiles/Schools/TX/Go...,Gold-Burg Independent School District District...,0.999955
4,4,4,Joaquin ISD,First,pdf,http://www.joaquinisd.net/upload/page/0025/Joa...,Joaquin Independent School District District o...,0.999955


In [7]:
texts_scraped_p.to_csv(os.path.join(os.getcwd(), '..', '..', 'data', 'text_scraped_p.csv'))

## Descriptive statistics

In [8]:
print("Predicted probability of DOI plan for first level documents:")
texts_scraped_p[['p_innovation']][texts_scraped_p.level == 'First'].mean()

Predicted probability of DOI plan for first level documents:


p_innovation    0.891641
dtype: float64

In [9]:
print("Predicted probability of DOI plan for second level and HTML documents:")
texts_scraped_p[['p_innovation']][texts_scraped_p.level != 'First'].mean()

Predicted probability of DOI plan for second level and HTML documents:


p_innovation    0.174182
dtype: float64

# Narrow Documents

Narrow documents to most likely DOI plan candidates. 

In [10]:
text_narrowed = texts_scraped_p.sort_values(['title', 'p_innovation'], ascending=False).drop_duplicates(subset=["title"], keep="first")

In [11]:
text_narrowed.to_csv(os.path.join(os.getcwd(), '..', '..', 'data', 'text_narrowed.csv'))

## Test performance

sample 30, label, what p are true doi plans?

In [12]:
retest_performance = False

In [13]:
if retest_performance:
    test = pd.read_csv(os.path.join(os.getcwd(), '..', '..', 'data', 'text_narrowed.csv'))
    test = test.sample(n=30)

In [14]:
test.head(30)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,title,level,type,link,text,p_innovation
33,2753,2753,2753,Wharton ISD,Second,pdf,http://whartonisd.net/upload/page/0449/WISD-Di...,Wharton ISD District of Innovation Plan 2018-2...,0.997414
660,2856,2856,2856,Celina ISD,Second,pdf,https://www.celinaisd.com/wp-documents/Require...,CISD District of Innovation Plan The Celina In...,0.250854
610,144,144,144,Covington ISD,First,pdf,http://www.covingtonisd.org/users/0004/docs/DO...,1 Covington ISD District of Innovation Plan Fe...,0.998519
444,2872,2872,2872,Humble ISD,Second,pdf,https://www.humbleisd.net/cms/lib/TX01001414/C...,District of Innovation APPROVED Humble ISD Boa...,0.999719
513,296,296,296,Gatesville ISD,First,pdf,https://1.cdn.edl.io/TWaRYIrtC8CtGlsG7gYXLr9bl...,Microsoft Word - GISD DOI PLAN - Final.docx GA...,0.999265
52,1571,1571,1571,Waelder ISD,Second,pdf,https://1.cdn.edl.io/9CRBFUkUbnGtYesJcLIn8dJoa...,WAELDER ISD: A DISTRICT PLAN FOR INNOVATION AN...,0.999872
79,72,72,72,Tolar ISD,First,pdf,http://www.tolarisd.org/cms/lib3/TX01000982/Ce...,Tolar Independent School District District Of ...,0.999085
590,3226,3226,3226,DeSoto ISD,Second,pdf,http://www.desotoisd.org/UserFiles/Servers/Ser...,Microsoft Word - DeSoto ISD DoI LocalPlan - am...,0.999955
229,113,113,113,Pettus ISD,First,pdf,https://1.cdn.edl.io/OoOdMkRMlHdkVt1cNv60y52IA...,PETTUS INDEPENDENT SCHOOL DISTRICT District of...,0.999875
639,3650,3650,307,Claude ISD,html,html,http://www.claudeisd.net/293354_4,Claude Independent School District - Local Inn...,0.999432


In [15]:
# Make first 30 documents easy to read.
for i, v in enumerate(test.loc[:,:]['text'][0:30]):
    print('\n')
    print('-'*30)
    print('DISTRICT {}'.format(i))
    print('-'*30)
    print('\n')
    print(v[0:4000])



------------------------------
DISTRICT 0
------------------------------


Wharton ISD District of Innovation Plan 2018-2019 through 2022-2023 Monica Rath Dean of Instruction, Sivells Elementary; Community Member Sheila Taylor Drill Team Instructor, Wharton High School; Community member, Business owner Donna Grissom District G/T Coordinator ; Community Member Ashley Ivy Math Instructional Facilitator, Wharton Elementary Pam Barnes Educational Aide, Wharton High School; Kim Williams Response to Intervention Teacher, Sivells Elementary; Community member, Parent Cindy Mahalitc Wharton ISD Director of Student Services, Federal Programs, and Testing Coordinator Lisa Kutach District Dyslexia Coordinator, Community member, Parent Fred Johnson Behavior Teacher, Sivells Elementary/Director of Wharton Boys’ and Girls’ Club Jeffrey Blair Community member Pat Blair Community member Gayle Parenica Executive Director, Curriculum, Instruction, and Accountability Wharton ISD 2017 District of Innovat

In [ ]:
ground_truth = [
    True, False, True, True, True,
    True, True, True, True, True,
    True, True, False, True, True,
    True, True, True, True, True,
    True, True, True, True, True,
    True, True, True, True, True
]

## Narrowed Descriptives

In [17]:
test = pd.read_csv(os.path.join(os.getcwd(), '..', '..', 'data', 'text_narrowed.csv'))

In [29]:
print('Average probability of DOI classification: ', test.p_innovation.mean())
print('SD of DOI classification: ',test.p_innovation.std())
print('Number of documents w/<.2 probability of DOI classification: ', len(test[test.p_innovation < .2]))

Average probability of DOI classification:  0.8482330004027946
SD of DOI classification:  0.3333338986361766
Number of documents w/<.2 probability of DOI classification:  103
